In [ ]:
import pandas as pd
import astropy.units as u
import warnings
from datetime import datetime
from lib.antenna import RT32
from lib.weather import Weather
from lib.spiral_sun_observation import SpiralSunObservation
from lib.position_generator import SpiralPositionGenerator
from lib.file_generator import PTFFileGenerator
from lib.data_processor import SolarDataProcessor
from lib.solar_plotter import SolarMapPlotter

# Imprimimos estadísticas resumidas del DataFrame final
pd.set_option('display.float_format', '{:.10f}'.format)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

e:\CODING\SpectropolarimeterMaps\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# RT32 location (Ventspils, Latvia)
rt32_antenna = RT32()
rt32_antenna.set_location(latitude=57.5535171694, longitude=21.8545525000, elevation=20)

# Define constants
year = 2025
month = 5
day = 8
hour_start = 9
minute_start = 15

# Set up weather parameters
temperature = u.Quantity(20.0, unit=u.deg_C)
pressure = u.Quantity(1013.25, unit=u.hPa)
relative_humidity = u.Quantity(60.0, unit=u.percent)
obswl = u.Quantity(50000, unit=u.nm) 

weather = Weather(temperature, pressure, relative_humidity, obswl)

# Create observation datetime
observation_datetime = datetime(year, month, day, hour_start, minute_start)

# Initialize the spiral sun observation
observation = SpiralSunObservation(weather, rt32_antenna, observation_datetime)

# File paths
fit_file_path = "data/examples/FITS/lnsp4_5ch_250508_091400_101010.fit"
ptf_file_path = "data/examples/PTF/sun_scan_250508_0915.ptf"
output_path = ""

# Calculate positions using the position generator
az_anten, el_anten, az_sun, el_sun, xx1, yy1, utc = observation.calculatePositions()

# Generate PTF file using the file generator
observation.generateFile(output_path, az_anten, el_anten, utc)

# Create sun position dataframe
sun_position_df = pd.DataFrame({'UTC': utc, 'SunX': xx1, 'SunY': yy1})

# Process FITS file using the data processor
print('-------------------------------------------------------------')
print('Start processing the FITS file:', fit_file_path)
band_processed_dfs = observation.data_processor.process_fits_file(fit_file_path, sun_position_df)

ObservationConfig 2025-05-08T09:15:00
-------------------------------------------------------------
Saved:  sun_scan_250508_0915.ptf    3300   points
-------------------------------------------------------------
Start processing the FITS file: real_data/lnsp4_5ch_250508_091400_101010.fit


NameError: name 'bintable_to_pandas' is not defined

In [ ]:
import matplotlib.pyplot as plt

# Use the SolarMapPlotter for visualization
# For quick visualization without saving maps, we can use matplotlib directly

# Extract data for 4.07GHz
data_4_07GHz = band_processed_dfs['4.07GHZ']

# Plot SunX vs SunY for 4.07GHz with uniform point size and red colormap
plt.figure(figsize=(8, 6))
scatter = plt.scatter(data_4_07GHz['SunX'], data_4_07GHz['SunY'], 
                     c=data_4_07GHz['STOKE_I_4.07GHZ'], 
                     s=50,  # Uniform point size
                     cmap='hot', alpha=0.9, label='4.07GHz')
plt.colorbar(scatter, label='STOKE_I_4.07GHZ')
plt.xlabel('SunX')
plt.ylabel('SunY')
plt.title('Sun Position for 4.07GHz with Uniform Point Size and Heatmap Colormap')
plt.legend()
plt.grid()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the sun position trajectory
plt.figure(figsize=(8, 6))
plt.plot(sun_position_df['SunX'], sun_position_df['SunY'], label='Sun Position')
plt.xlabel('SunX')
plt.ylabel('SunY')
plt.title('Sun Position Plot')
plt.legend()
plt.grid()
plt.show()

# Add a new cell to demonstrate the solar map creation
# Uncomment this section to generate solar maps
"""
# Create directory for saving images
images_output_path = "output/images"

# Generate solar maps using RBF interpolation
output_dir = observation.plotter.create_solar_maps_rbf(band_processed_dfs, images_output_path)
print(f"Solar maps saved to: {output_dir}")

# Alternatively, generate maps using Gaussian filtering
# output_dir = observation.createImages_GAUSS(fit_file_path, images_output_path)
"""